In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
143,This is one of the few episodes (if not the on...,negative
651,Saying this movie is extremely hard to follow ...,negative
451,"I just saw this film last night, and I have to...",positive
39,I am a big fan of Fred MacMurray and Carole Lo...,negative
749,"Ya know, I have no idea how everybody else's t...",negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\USER\AppData\Local\Temp\ipykernel_15552\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
143,one episode if one indisputable error storytel...,negative
651,saying movie extremely hard follow frustrating...,negative
451,saw film last night say loved every minute tak...,positive
39,big fan fred macmurray carole lombard and addi...,negative
749,ya know idea everybody else s teenage life wa ...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
negative    261
positive    239
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
143,one episode if one indisputable error storytel...,0
651,saying movie extremely hard follow frustrating...,0
451,saw film last night say loved every minute tak...,1
39,big fan fred macmurray carole lombard and addi...,0
749,ya know idea everybody else s teenage life wa ...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [11]:
print("y_test unique:", set(y))



y_test unique: {0, 1}


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
print("y_test unique:", set(y_test))
print("y_pred unique:", set(y_train))
print("y_test dtype:", y_test.dtype)


y_test unique: {0, 1}
y_pred unique: {0, 1}
y_test dtype: int64


In [ ]:
# import dagshub

# mlflow.set_tracking_uri('https://dagshub.com/.......')
# dagshub.init(repo_owner='anki...', repo_name='MLOps-NLP-Project', mlflow=True)
# # mlflow.set_experiment("Logistic Regression Baseline")
# mlflow.set_experiment("Logistic Regression Baseline")


import os
import dagshub
import mlflow
from dotenv import load_dotenv

load_dotenv()

dagshub.init(
    repo_owner=os.getenv("DAGSHUB_USER"),
    repo_name=os.getenv("DAGSHUB_REPO"),
    mlflow=True
)

mlflow.set_experiment("Logistic Regression Baseline")




Accessing as ankit-raj00

Initialized MLflow to track repo "ankit-raj00/MLOps-NLP-Project"

Repository ankit-raj00/MLOps-NLP-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/47377199e92c46bf85afb404331b3c7d', creation_time=1767534828513, experiment_id='0', last_update_time=1767534828513, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [15]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(
        sk_model=model,
        name="logistic_regression_model"
        )

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-04 20:14:02,323 - INFO - Starting MLflow run...
2026-01-04 20:14:03,226 - INFO - Logging preprocessing parameters...
2026-01-04 20:14:04,323 - INFO - Initializing Logistic Regression model...
2026-01-04 20:14:04,323 - INFO - Fitting the model...
2026-01-04 20:14:04,389 - INFO - Model training complete.
2026-01-04 20:14:04,390 - INFO - Logging model parameters...
2026-01-04 20:14:04,776 - INFO - Making predictions...
2026-01-04 20:14:04,780 - INFO - Calculating evaluation metrics...
2026-01-04 20:14:04,805 - INFO - Logging evaluation metrics...
2026-01-04 20:14:06,565 - INFO - Saving and logging the model...
2026-01-04 20:14:28,226 - INFO - Model training and logging completed in 25.00 seconds.
2026-01-04 20:14:28,230 - INFO - Accuracy: 0.6
2026-01-04 20:14:28,231 - INFO - Precision: 0.5507246376811594
2026-01-04 20:14:28,232 - INFO - Recall: 0.6666666666666666
2026-01-04 20:14:28,234 - INFO - F1 Score: 0.6031746031746031


🏃 View run thoughtful-deer-53 at: https://dagshub.com/ankit-raj00/MLOps-NLP-Project.mlflow/#/experiments/0/runs/1f4f62b0cc2c4440a2beff5bad347a99
🧪 View experiment at: https://dagshub.com/ankit-raj00/MLOps-NLP-Project.mlflow/#/experiments/0
